In [1]:
%%capture
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu128

In [2]:
%%capture
!pip install scikit-image scikit-learn scipy numpy matplotlib pillow

In [3]:
from skimage.segmentation import slic
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import lars_path
from scipy.stats import kendalltau, spearmanr

def lime_explanation(model, image, predicted_class, num_samples=5000, num_features=10):    
    segments = slic(image, n_segments=100, compactness=10, sigma=1)
    num_superpixels = len(np.unique(segments))
    
    z_perturbations = []
    z_predictions = []
    z_weights = []
    
    x_prime_original = np.ones(num_superpixels)
    
    for _ in range(num_samples):
        num_on = np.random.randint(1, num_superpixels + 1)
        on_indices = np.random.choice(num_superpixels, num_on, replace=False)
        
        z_prime = np.zeros(num_superpixels)
        z_prime[on_indices] = 1
        
        z_i = image.copy()
        for sp_idx in range(num_superpixels):
            if z_prime[sp_idx] == 0:
                z_i[segments == sp_idx] = 0
        
        z_i_pil = Image.fromarray(z_i.astype('uint8'))
        z_i_tensor = preprocess(z_i_pil).unsqueeze(0)
        if torch.cuda.is_available():
            z_i_tensor = z_i_tensor.to('cuda')
        
        with torch.no_grad():
            pred = model(z_i_tensor)
        pred_score = pred[0, predicted_class].item()
        
        distance = np.sqrt(np.sum((z_prime - x_prime_original) ** 2))
        kernel_width = 0.25 * np.sqrt(num_superpixels)
        weight = np.exp(-(distance ** 2) / (kernel_width ** 2))
        
        z_perturbations.append(z_prime)
        z_predictions.append(pred_score)
        z_weights.append(weight)
    
    z_perturbations = np.array(z_perturbations)
    z_predictions = np.array(z_predictions)
    z_weights = np.array(z_weights)
    
    lasso = Lasso(alpha=0.01, max_iter=5000)
    lasso.fit(z_perturbations, z_predictions, sample_weight=z_weights)
    
    feature_importance = np.abs(lasso.coef_)
    top_k_indices = np.argsort(feature_importance)[-num_features:]
    
    z_topk = z_perturbations[:, top_k_indices]
    final_model = LinearRegression()
    final_model.fit(z_topk, z_predictions, sample_weight=z_weights)
    
    w = np.zeros(num_superpixels)
    w[top_k_indices] = final_model.coef_
    
    return w, segments

def visualize_explanation_lime(image_np, coef, segments, num_features=10):
    top_k_indices = np.argsort(np.abs(coef))[-num_features:]
    explanation = np.zeros(segments.shape)
    for i in top_k_indices:
        explanation[segments == i] = coef[i]
    explanation = (explanation - explanation.min()) / (explanation.max() - explanation.min() + 1e-10)
    fig, axes = plt.subplots(4, 1, figsize=(4, 16))
    axes[0].imshow(image_np)
    axes[0].axis('off')
    axes[1].imshow(segments)
    axes[1].axis('off')
    axes[2].imshow(explanation, cmap='hot')
    axes[2].axis('off')
    axes[3].imshow(image_np)
    axes[3].imshow(explanation, cmap='hot', alpha=0.7)
    axes[3].axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0)
    return fig

In [4]:
def smoothgrad_explanation(model, image_np, predicted_class, num_samples=50, noise_level=0.15):
    image_pil = Image.fromarray(image_np.astype('uint8'))
    image_tensor = preprocess(image_pil).unsqueeze(0)
    if torch.cuda.is_available():
        image_tensor = image_tensor.to('cuda')
    
    all_gradients = []
    
    for _ in range(num_samples):
        noise = torch.randn_like(image_tensor) * noise_level
        noisy_image = image_tensor + noise
        noisy_image.requires_grad = True
        output = model(noisy_image)
        score = output[0, predicted_class]
        model.zero_grad()
        score.backward()
        gradient = noisy_image.grad.data.cpu().numpy()[0]
        all_gradients.append(gradient)
    
    smoothed_gradients = np.mean(all_gradients, axis=0)
    
    smoothed_gradients = np.abs(smoothed_gradients).mean(axis=0)
    
    return smoothed_gradients
    
def visualize_smoothgrad(image_np, smoothed_gradients):
    if smoothed_gradients.ndim == 3:
        smoothed_gradients = smoothed_gradients.mean(axis=-1)
    smoothed_gradients = (smoothed_gradients - smoothed_gradients.min()) / (
        smoothed_gradients.max() - smoothed_gradients.min() + 1e-10
    )

    fig, axes = plt.subplots(3, 1, figsize=(4, 12))
    for ax in axes:
        ax.axis('off')

    axes[0].imshow(image_np)
    axes[1].imshow(smoothed_gradients, cmap='hot')
    axes[2].imshow(image_np)
    axes[2].imshow(smoothed_gradients, cmap='hot', alpha=0.7)

    plt.subplots_adjust(0, 0, 1, 1, 0, 0)
    return fig


In [5]:
def compute_ranking_correlations_kendall(lime_coef, smoothgrad, segments):
    lime_importance = np.abs(lime_coef)
    
    num_superpixels = len(np.unique(segments))
    smoothgrad_per_superpixel = np.zeros(num_superpixels)
    
    for i in range(num_superpixels):
        mask = (segments == i)
        smoothgrad_per_superpixel[i] = np.mean(smoothgrad[mask])
    valid_indices = (lime_importance > 0) | (smoothgrad_per_superpixel > 0)
    lime_valid = lime_importance[valid_indices]
    smoothgrad_valid = smoothgrad_per_superpixel[valid_indices]
    
    kendall_tau, kendall_p = kendalltau(lime_valid, smoothgrad_valid)
    return kendall_tau, kendall_p

def compute_ranking_correlations_spearman(lime_coef, smoothgrad, segments):
    lime_importance = np.abs(lime_coef)
    
    num_superpixels = len(np.unique(segments))
    smoothgrad_per_superpixel = np.zeros(num_superpixels)
    
    for i in range(num_superpixels):
        mask = (segments == i)
        smoothgrad_per_superpixel[i] = np.mean(smoothgrad[mask])
    valid_indices = (lime_importance > 0) | (smoothgrad_per_superpixel > 0)
    lime_valid = lime_importance[valid_indices]
    smoothgrad_valid = smoothgrad_per_superpixel[valid_indices]
    
    spearman_rho, spearman_p = spearmanr(lime_valid, smoothgrad_valid)
    return spearman_rho, spearman_p

In [7]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import json
import os 

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model.eval()  # Set model to evaluation mode

# Define the image preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]   
    )
])

preprocess_for_display = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

lime_figs = []
smoothgrad_figs = []

# Load the ImageNet class index mapping
with open("cs521-4-2-imagenet_class_index.json") as f:
    class_idx = json.load(f)
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
idx2synset = [class_idx[str(k)][0] for k in range(len(class_idx))]
id2label = {v[0]: v[1] for v in class_idx.values()}

imagenet_path = './cs521-4-2-imagenet_samples'

# List of image file paths
image_paths = os.listdir(imagenet_path)


for img_path in image_paths:
    if not img_path.endswith('.JPEG'):
        continue
    # Open and preprocess the image
    # my_img = os.path.join(img_path, os.listdir(img_path)[2])
    my_img = os.path.join(imagenet_path, img_path)
    input_image = Image.open(my_img).convert('RGB')
    input_tensor = preprocess(input_image)
    image_np = np.transpose(input_tensor.numpy(), (1, 2, 0))
    input_tensor_for_display = preprocess_for_display(input_image)
    image_np_for_display = np.transpose(input_tensor_for_display.numpy(), (1, 2, 0))
    
    input_batch = input_tensor.unsqueeze(0)  # Create a mini-batch as expected by the model

    # Move the input and model to GPU if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    # Perform inference
    with torch.no_grad():
        output = model(input_batch)

    # Get the predicted class index
    _, predicted_idx = torch.max(output, 1)
    predicted_idx = predicted_idx.item()
    predicted_synset = idx2synset[predicted_idx]
    predicted_label = idx2label[predicted_idx]

    print(f"Predicted label: {predicted_synset} ({predicted_label})")
    print("Generating LIME explanation...")
    coef, segments = lime_explanation(model, image_np, predicted_idx, num_samples=500, num_features=50)
    fig = visualize_explanation_lime(image_np_for_display, coef, segments, num_features=10)
    lime_figs.append(fig)
    plt.savefig(f'lime_explanation_{img_path}')
    plt.close()
    print(f"Explanation saved to lime_explanation_{img_path}\n")
    print("Generating SmoothGrad explanation...")
    smoothgrad = smoothgrad_explanation(model, image_np, predicted_idx, num_samples=50, noise_level=0.15)
    fig_sg = visualize_smoothgrad(image_np_for_display, smoothgrad)
    plt.savefig(f'smoothgrad_explanation_{img_path}')
    smoothgrad_figs.append(fig_sg)
    plt.close()
    print(f"SmoothGrad explanation saved")    
    print("Computing ranking correlations...")
    kendall_tau, kendall_p = compute_ranking_correlations_kendall(coef, smoothgrad, segments)
    spearman_rho, spearman_p = compute_ranking_correlations_spearman(coef, smoothgrad, segments)    
    print(f"Kendall-Tau: {kendall_tau:.4f} (p={kendall_p:.4f})")
    print(f"Spearman Rho: {spearman_rho:.4f} (p={spearman_p:.4f})")
    

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted label: n03792782 (mountain_bike)
Generating LIME explanation...
Explanation saved to lime_explanation_mountain_bike.JPEG

Generating SmoothGrad explanation...
SmoothGrad explanation saved
Computing ranking correlations...
Kendall-Tau: -0.1141 (p=0.1797)
Spearman Rho: -0.1624 (p=0.1858)
Predicted label: n03250847 (drumstick)
Generating LIME explanation...


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Explanation saved to lime_explanation_mousetrap.JPEG

Generating SmoothGrad explanation...
SmoothGrad explanation saved
Computing ranking correlations...
Kendall-Tau: 0.2122 (p=0.0099)
Spearman Rho: 0.3383 (p=0.0032)


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


...

/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Explanation saved to lime_explanation_peacock.JPEG

Generating SmoothGrad explanation...
SmoothGrad explanation saved
Computing ranking correlations...
Kendall-Tau: -0.1233 (p=0.1518)
Spearman Rho: -0.1820 (p=0.1435)


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Predicted label: n03938244 (pillow)
Generating LIME explanation...
Explanation saved to lime_explanation_piggy_bank.JPEG

Generating SmoothGrad explanation...
SmoothGrad explanation saved
Computing ranking correlations...
Kendall-Tau: 0.0913 (p=0.2617)
Spearman Rho: 0.1459 (p=0.2085)


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
